In [ ]:
%matplotlib notebook

import cmapPy.pandasGEXpress.parse

import cmapPy.math.fast_corr

import plotly.express as pltexpr

import sklearn.cluster

import pandas as pd

import numpy

from matplotlib import pyplot as plt

from scipy.cluster.hierarchy import dendrogram

import scipy.cluster.hierarchy

import json

In [ ]:
gctoo = cmapPy.pandasGEXpress.parse.parse("2020_Q3_Achilles_CCLE_expression_r19144x1305.gctx")
print(gctoo)

In [ ]:
numpy_df = gctoo.data_df.to_numpy()

result = cmapPy.math.fast_corr.fast_corr(numpy_df)

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = numpy.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = numpy.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)
    
    print(linkage_matrix)

    # Plot the corresponding dendrogram
    r_dict = dendrogram(linkage_matrix, **kwargs)

    
    
    return linkage_matrix, list(map(int, r_dict["ivl"])), r_dict["leaves"]

# from https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

In [ ]:
fig, axs = plt.subplots(3,1)

In [ ]:
AffinityProp = sklearn.cluster.AffinityPropagation(affinity = "precomputed").fit(result)

print("ran")
print(AffinityProp)

In [ ]:
result_df = pd.DataFrame(result)

labels_df = pd.DataFrame(AffinityProp.labels_)
labels_df.rename(columns = {0:1305}, inplace = True)

transposed_labels_df = labels_df.transpose()

transposed_labels_df.insert(1305, 1305, 20000)

print(result_df)
print(labels_df)
print(transposed_labels_df)

column_label_df = result_df.join(labels_df)

label_df = column_label_df.append(transposed_labels_df, ignore_index=True)

print(label_df)

col_sorted = label_df.sort_values(1305)

sorted_df = col_sorted.sort_values(1305, axis = 1)

print(col_sorted)
print(sorted_df)

sorted_df = sorted_df.drop(1305, axis = 0)
sorted_df = sorted_df.drop(1305, axis = 1)

print(sorted_df)



In [ ]:
cluster_centers = []

print(AffinityProp.cluster_centers_indices_)
for i in range (0, len(AffinityProp.cluster_centers_indices_)):
    cluster_centers.append(result[AffinityProp.cluster_centers_indices_[i]])
    

In [ ]:
ward_super_clusters = sklearn.cluster.AgglomerativeClustering(affinity = "euclidean", n_clusters = 71, linkage = "average", compute_full_tree = True, compute_distances = True).fit(cluster_centers)

In [ ]:
super_linkage_matrix, super_dendro_labels, super_leaves = plot_dendrogram(ward_super_clusters, truncate_mode='level', p=51, distance_sort = "ascending", ax=axs[0])
plt.show()

In [ ]:
print(super_dendro_labels)

super_dendro_labels_index = []

for i in range(0, len(super_dendro_labels)):
    super_dendro_labels_index.append(super_dendro_labels.index(i))
    

print(super_dendro_labels_index)
print(AffinityProp.cluster_centers_indices_)

In [ ]:
print(label_df)

#list comperhension 
#then add col


for i in range(0, 1305):
    val = label_df.loc[i, (1305)]
    label_df.loc[i, (1305)] = super_dendro_labels_index[val]
    label_df.loc[1305, (i)] = super_dendro_labels_index[val]
    
    
super_col_sorted = label_df.sort_values(1305)

super_sorted_df = super_col_sorted.sort_values(1305, axis = 1)

print(super_col_sorted)
print(super_sorted_df)

super_sorted_df = super_sorted_df.drop(1305, axis = 0)
super_sorted_df = super_sorted_df.drop(1305, axis = 1)

print(super_sorted_df)

In [ ]:
current_sub = 59

In [ ]:
sorted_with_labels = super_col_sorted.sort_values(1305, axis = 1)

sub_cluster = sorted_with_labels.loc[sorted_with_labels[1305] == current_sub]
print(sub_cluster)
ward_sub_cluster = sklearn.cluster.AgglomerativeClustering(affinity = "euclidean", n_clusters = len(sub_cluster)-1, linkage = "average", compute_full_tree = True, compute_distances = True).fit(sub_cluster)

In [ ]:
sub_linkage_matrix, sub_dendro_labels, sub_leaves = plot_dendrogram(ward_sub_cluster, truncate_mode='level', p=51, distance_sort = "ascending", ax= axs[1])
plt.show()

In [ ]:
df_super_linkage_matrix = pd.DataFrame(super_linkage_matrix)
print(df_super_linkage_matrix)


for row in df_super_linkage_matrix.iterrows():
    print(row)

In [ ]:
df_sub_linkage_matrix = pd.DataFrame(sub_linkage_matrix)
print(df_sub_linkage_matrix)

In [ ]:
where_col = 5
not_where = 5

try:
    where = df_super_linkage_matrix.loc[df_super_linkage_matrix[0] == current_sub]
    where_col = 0
    not_where = 1
    print(where.index[0])
except IndexError:
    where = df_super_linkage_matrix.loc[df_super_linkage_matrix[1] == current_sub]
    where_col = 1
    not_where = 0

print(where.index[0])
print(where.index.to_numpy()[0])

print(where)

In [ ]:
n_super_samples = len(df_super_linkage_matrix) + 1

In [ ]:
print(df_sub_linkage_matrix) 

In [ ]:
n_sub_samples = len(df_sub_linkage_matrix) + 1
new_location = where.index[0]
new_cluster = n_super_samples - 1

print(n_sub_samples)
print(n_super_samples)
print(new_cluster)

for i in range(0, n_sub_samples-1):
    zero_leaf = df_sub_linkage_matrix[0][i]
    one_leaf = df_sub_linkage_matrix[1][i]
    if zero_leaf >= n_sub_samples:
        #something to change it so it can be added to linkage_matrix
        df_sub_linkage_matrix[0][i] = zero_leaf + n_super_samples  -1  + new_location
        
    else:
        #df_sub_linkage_matrix[0][i] = sub_cluster.index[int(zero_leaf)]
        df_sub_linkage_matrix[0][i] = new_cluster
        new_cluster += 1
        
    if one_leaf >= n_sub_samples:
        #something to change it so it can be added to linkage_matrix
        df_sub_linkage_matrix[1][i] = one_leaf  + n_super_samples - 1 + new_location
        
    else:
        #df_sub_linkage_matrix[1][i] = sub_cluster.index[int(one_leaf)]
        df_sub_linkage_matrix[1][i] = new_cluster
        new_cluster += 1
        

        

        
#set the greatest value = to 1
#that is the sample name that was replaced so this is done to aviod dupes

for i in range(0, len(df_sub_linkage_matrix)):
    if df_sub_linkage_matrix.loc[i, 0] == n_super_samples - 1:
        df_sub_linkage_matrix.at[i, 0] = current_sub

    if df_sub_linkage_matrix.loc[i, 1] == n_super_samples - 1:
        df_sub_linkage_matrix.at[i, 1] = current_sub

        
        
print(df_sub_linkage_matrix) 


In [ ]:
df_sub_linkage_matrix.index = df_sub_linkage_matrix.index + new_location
print(df_sub_linkage_matrix) 

In [ ]:
print(df_sub_linkage_matrix.index[0])
print(where.loc[where.index[0], 0])


where.at[new_location, where_col] =  new_location + n_super_samples +  (2 * n_sub_samples) - 3

    
            
if where.loc[new_location, not_where] > n_super_samples:
        if where.loc[new_location, not_where] - n_super_samples < new_location:
            where.at[new_location, not_where] = where.loc[new_location, not_where] + n_sub_samples - 1
        else:
            where.at[new_location, not_where] = where.loc[new_location, not_where] + (2 * n_sub_samples) - 2

where.at[new_location, 3] =  where.loc[new_location, 3] + n_sub_samples

#a really dumb way to rename the index
#.setindex needs a col to set as the index 
total_sub = pd.DataFrame(where.to_numpy(), index = [where.index[0] + n_sub_samples-1])

print(where)
print(total_sub)

In [ ]:
total_sub = df_sub_linkage_matrix.append(total_sub)

In [ ]:
for row in df_super_linkage_matrix.iterrows():
    print(row)

In [ ]:
adding = 0

for i in range(0, n_super_samples-1):
    zero_leaf = df_super_linkage_matrix[0][i]
    one_leaf = df_super_linkage_matrix[1][i]

    if zero_leaf >+ n_super_samples:
        if zero_leaf - n_super_samples < new_location:
            df_super_linkage_matrix[0][i] = df_super_linkage_matrix[0][i] + n_sub_samples - 1
        else:
            df_super_linkage_matrix[0][i] = df_super_linkage_matrix[0][i] + (2 * n_sub_samples) - 2
 
    if one_leaf >= n_super_samples:
        if one_leaf - n_super_samples < new_location:
            df_super_linkage_matrix[1][i] = df_super_linkage_matrix[1][i] + n_sub_samples - 1
        else:
            df_super_linkage_matrix[1][i] = df_super_linkage_matrix[1][i] + (2 * n_sub_samples) - 2
        

In [ ]:
print(df_super_linkage_matrix)

for row in df_super_linkage_matrix.iterrows():
    print(row)

In [ ]:
df_super_linkage_matrix.drop(new_location, inplace = True)


df_super_linkage_matrix.index = list(range(0, new_location)) + list(range(new_location + 1 +n_sub_samples , len(df_super_linkage_matrix)+1 + n_sub_samples))
    
    
for row in df_super_linkage_matrix.iterrows():
    print(row)

In [ ]:
for row in total_sub.iterrows():
    print(row)

In [ ]:
    
new_linkage_matrix = df_super_linkage_matrix.append(total_sub)
new_linkage_matrix.sort_index(inplace = True)

In [ ]:
print(new_linkage_matrix)
print(new_linkage_matrix[0][17])

In [ ]:
for row in new_linkage_matrix.iterrows():
    print(row)

In [ ]:
#one of two checks that my code often fails, see which row causes the failure 

#_check_hierarchy_uses_cluster_before_formed

Z = numpy.asarray(new_linkage_matrix, order='c')

n = Z.shape[0] + 1
for i in range(0, n - 1):
    if Z[i, 0] >= n + i or Z[i, 1] >= n + i:
        print(i)
        print(Z[i, 0])
    else:
        print(False)

In [ ]:
#one of two checks that my code often fails, see which row causes the failure 

#check_hierarchy_uses_cluster_more_than_once

n = Z.shape[0] + 1
chosen = set([])
for i in range(0, n - 1):
    if (Z[i, 0] in chosen) or (Z[i, 1] in chosen) or Z[i, 0] == Z[i, 1]:
        print(i)
        print(Z[i, 0])
    chosen.add(Z[i, 0])
    chosen.add(Z[i, 1])
    print(False)

print(chosen)

for i in range(0, 146):
    if i not in chosen:
        print(i)

In [ ]:
scipy.cluster.hierarchy.is_valid_linkage(new_linkage_matrix)

In [ ]:
r_dict = dendrogram(new_linkage_matrix, truncate_mode='level', p=51, distance_sort = "ascending", ax= axs[2])
plt.show()